<a href="https://colab.research.google.com/github/manu9418/IRH/blob/main/Evaporacion_ChowVenTe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *TAREA - METODOS DE ANALISIS*

Calculo de la evaporacion  
Ref. Hidrologia Aplicada. Ven Te Chow  
Eduardo Chavarri V. SEPT - 2021  
Manuel Figueroa Culqui 2021-II


# Importando libs necesarias

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# # Cargar datos desde drive
# from google.colab import drive
# drive.mount('/content/gdrive')

# fpath = '/content/gdrive/MyDrive/Data/data_evap.xlsx'
# xls = pd.ExcelFile(fpath)
# df = pd.read_excel(xls, header=None, names= ['latitud (°)', 'altitud (msnm)', 'anho', 'mes', 'dia', 'radiacion neta (MJm-2d-1)', 'radiacion onda corta (MJm-2d-1)', 'radiacion extraterrestre (MJm-2d-1)', 'temperatura media (°C)', 'temperatura punto rocio (°C)', 'temperatura minima (°C)', 'temperatura maxima (°C)', 'horas sol (hrs)', 'albedo (%)', 'humedad relativa (%)', 'velocidad viento 2m (ms-1)'])
# df

Mounted at /content/gdrive


# Datos de prueba

In [ ]:
# Cargar datos de prueba
data0 = np.array([[-12,	500,	2021,	12,	19,	17,	23.45,	27.9,	20,	5,	4,	35,	9,	0.08,	80,	2.5]])
df = pd.DataFrame(data=data0, columns= ['latitud (°)', 'altitud (msnm)', 'anho', 'mes', 'dia', 'radiacion neta (MJm-2d-1)', 'radiacion onda corta (MJm-2d-1)', 'radiacion extraterrestre (MJm-2d-1)', 'temperatura media (°C)', 'temperatura punto rocio (°C)', 'temperatura minima (°C)', 'temperatura maxima (°C)', 'horas sol (hrs)', 'albedo (%)', 'humedad relativa (%)', 'velocidad viento 2m (ms-1)'])
df

,latitud (°),altitud (msnm),anho,mes,dia,radiacion neta (MJm-2d-1),radiacion onda corta (MJm-2d-1),radiacion extraterrestre (MJm-2d-1),temperatura media (°C),temperatura punto rocio (°C),temperatura minima (°C),temperatura maxima (°C),horas sol (hrs),albedo (%),humedad relativa (%),velocidad viento 2m (ms-1)
0,-12.0,500.0,2021.0,12.0,19.0,17.0,23.45,27.9,20.0,5.0,4.0,35.0,9.0,0.08,80.0,2.5


In [ ]:
"""
Functions for converting dates to/from JD and MJD. Assumes dates are historical dates, including the transition from the Julian calendar to the Gregorian calendar in 1582. 
No support for proleptic Gregorian/Julian calendars.
:Author: Matt Davis
:Website: http://github.com/jiffyclub
"""

def date_to_jd(year,month,day):
    """
    Convert a date to Julian Day.
    
    Algorithm from 'Practical Astronomy with your Calculator or Spreadsheet', 
        4th ed., Duffet-Smith and Zwart, 2011.
    
    Parameters
    ----------
    year : int
        Year as integer. Years preceding 1 A.D. should be 0 or negative.
        The year before 1 A.D. is 0, 10 B.C. is year -9.
        
    month : int
        Month as integer, Jan = 1, Feb. = 2, etc.
    
    day : float
        Day, may contain fractional part.
    
    Returns
    -------
    jd : float
        Julian Day
        
    Examples
    --------
    Convert 6 a.m., February 17, 1985 to Julian Day
    
    >>> date_to_jd(1985,2,17.25)
    2446113.75
    
    """
    import math
    if month == 1 or month == 2:
        yearp = year - 1
        monthp = month + 12
    else:
        yearp = year
        monthp = month
    
    # this checks where we are in relation to October 15, 1582, the beginning
    # of the Gregorian calendar.
    if ((year < 1582) or
        (year == 1582 and month < 10) or
        (year == 1582 and month == 10 and day < 15)):
        # before start of Gregorian calendar
        B = 0
    else:
        # after start of Gregorian calendar
        A = math.trunc(yearp / 100.)
        B = 2 - A + math.trunc(A / 4.)
        
    if yearp < 0:
        C = math.trunc((365.25 * yearp) - 0.75)
    else:
        C = math.trunc(365.25 * yearp)
        
    D = math.trunc(30.6001 * (monthp + 1))
    
    jd = B + C + D + day + 1720994.5
    
    return jd

# import datetime
# my_date = datetime.date(2021,12,19)   
# my_date.toordinal() + 1721424.5

In [ ]:
# Informacion de entrada
lat=df.iloc[:,0].values[0];     # Latitud
alt=df.iloc[:,1].values[0];     # Altitud [msnm]
ano=df.iloc[:,2].values[0];     # Anho
mes=df.iloc[:,3].values[0];     # Mes
dia=df.iloc[:,4].values[0];     # Dia
Rn=df.iloc[:,5].values[0];      # Radiacion neta [MJm**-2d**-1]
Roc=df.iloc[:,6].values[0];     # Radiacion onda corta [MJm**-2d**-1]
Rext=df.iloc[:,7].values[0];    # Radiacion extraterrestre [MJm**-2d**-1]
temp=df.iloc[:,8].values[0];    # Temperatura media [°C]
Tpr=df.iloc[:,9].values[0];     # Temperatura Punto Rocio [°C)
Tmin=df.iloc[:,10].values[0];   # Temperatura minima [°C]
Tmax=df.iloc[:,11].values[0];   # Temperatura maxima [°C]
n_sol=df.iloc[:,12].values[0];  # Horas sol [h]
albedo=df.iloc[:,13].values[0]; # Albedo [%]
hr=df.iloc[:,14].values[0];     # Humedad relativa [%]
vv=df.iloc[:,15].values[0];     # velocidad del viento a 2.0 de altura [ms**-1]

In [ ]:
def comp_e(t):
  """
  Presion de vapor de saturacion con respecto al agua liquida [mb o hPa]
  Variacion Tetens (1930) propuesta por Bolton (1980) disponible en https://www.yumpu.com/en/document/read/53348819/tetens-equation 
  ec. Clausius-Clapeyron 1977 con T en °C y mb/hPa 
  """
  e = 6.108*np.exp((17.27*t)/(237.3+t)) # mb o hPa
  return e

In [ ]:
# Calculo energia disponible para la evaporacion
def comp_evap(lat=lat, alt=alt, ano=ano, mes=mes, dia=dia, Rn=Rn, Roc=Roc, Rext=Rext, 
              temp=temp, Tpr=Tpr, Tmin=Tmin, Tmax=Tmax, n_sol=n_sol, albedo=albedo, hr=hr, vv=vv):

  nj= date_to_jd(ano,mes,dia);                                                     # Numero juliano (J) []
  dec_sol=0.4093*np.sin((2*np.pi*nj/365)-1.405);                                   # Angulo de declinacion solar (δ) [rad]
  lat_rad=lat*np.pi/180;                                                           # Latitud en radianes (φ) [rad]
  ang_atard=np.arccos(-np.tan(lat_rad)*np.tan(dec_sol));                           # Angulo de horas de atardecer (Ws) [rad]
  max_N= 24*ang_atard/np.pi;                                                       # Maximas horas de luz posible (N) [hrs]
  dist_sol= 1+0.033*np.cos(2*np.pi*nj/365);                                        # Distancia relativa entre la tierra y el sol (dr) []
  rad_ext=15.392*dist_sol*(ang_atard*np.sin(lat_rad)*np.sin(dec_sol)+np.cos(lat_rad)*np.cos(dec_sol)*np.sin(ang_atard)); # Maxima radiacion solar extraterrestre (So) [mm/dia]

  import sys
  prompt_01 = 'Dispone de la radiacion neta (MJm**-2d**-1) SI(1), NO (2) ? : ';
  resp=int(input(prompt_01));

  if resp == 1:
    print('Rn -> Rn_mmxd')
    Rn_mmxd=Rn/(2.501-0.002361*temp);                    # Rn/lv [ MJm**-2d**-1 / MJKg**-1 ] = [mm/dia]
  elif resp==2:
    prompt_01 = 'Dispone de la radiacion solar de onda corta (MJm**-2d**-1) SI(1), NO (2) ? ';
    resp=int(input(prompt_01)); 
    if resp == 1:
      print('Roc -> St(rad_inc_oc) -> Rnoc')
      rad_inc_oc=Roc/(2.501-0.002361*temp)                       # Roc/lv [MJm**-2d**-1 / MJKg**-1 ] = [mm/dia] equivalentes de agua
      # print('rad_inc_oc', rad_inc_oc)
      rad_noc=rad_inc_oc*(1-albedo);                             # Radiacion neta de onda corta (Sn), ver valores de albedo en tablas
      # print('rad_noc',rad_noc)
      prompt_01 = 'Dispone de la temperatura al punto de rocio SI(1), NO (2) ? ';
      resp=int(input(prompt_01));
      if resp == 1:
        print('Tpr -> e -> emisividad neta')
        pvap=comp_e(Tpr)/10;          #en kPa            # Presion de vapor de saturacion con respecto al agua liquida [mb o hPa] input: T en °C 
        emi_neta=0.34-0.14*np.sqrt(pvap);                # Emisividad neta
        # print('emi_neta=', emi_neta)
      elif resp==2:
        prompt_01 = 'Dispone de la temperatura minima SI(1), NO (2) ? ';
        resp=int(input(prompt_01)); 
        if resp == 1:
          print('tmin -> emisividad neta')
          emi_neta=0.34-0.14*np.sqrt(comp_e(Tmin)/10);       # Emisividad neta en funcion de tmin  
        elif resp==2:
          print('tmedia -> emisividad neta')
          emi_neta=-0.02+0.261*np.exp(-7.77e-4*temp**2);     # Emisividad neta en funcion de la temp media del aire (°C)

        # Coeficientes de nubosidad
        # ac y bc: ac=1.35, bc=-0.35 (areas aridas)
        #          ac=1.0,  bc=0.0   (areas humedas)
        # prompt_01 = 'El area de estudio es arida(1) o humeda (2) ? ';
        # areash=int(input(prompt_01));
        # if areash == 1:
        #   ac=1.35;
        #   bc=-0.35;
        # elif areash == 2:
        #   ac=1;
        #   bc=0;
        # print('rad para cielo claro -> f_nubosidad + emi_neta --> rnol +rnoc --> radmxd')
        # rad_sol_cc=0.25*rad_ext*(2.501-0.002361*temp);                    # Radiacion solar para cielo claro (St0 = as*So) [MJm**-2d**-1]
                                                                          # as=0.25, So = So(rad solar esxtrat)*(lv) 
        # rad_inc_oc=(0.25+0.5*n_sol/max_N)*rad_ext*(2.501-0.002361*temp);  # Radiacion solar incidente de onda corta (St) [MJm**-2d**-1]
        # f_nubosidad=ac*(rad_inc_oc/rad_sol_cc)+bc;                        # Factor de nubosidad (f)
        # rn_ol=-f_nubosidad*emi_neta*4.903e-9*(temp+273.2)**4;             # Radiacion neta de onda larga (Ln) [MJm**-2d**-1]
        # Rn = rad_noc+rn_ol                                                # Radiacion neta (Rn) [mm/dia]
        # Rn_mmxd= Rn/(2.501-0.002361*temp);                                # Rn/lv [ MJm**-2d**-1 / MJKg**-1 ] = [mm/dia]

    elif resp==2:
      prompt_01 = 'Dispone de registros de horas de sol SI(1), NO (2) ? ';
      resp=int(input(prompt_01));
      if resp == 1:
        print('St(rad_inc_oc) -> Rnoc')
        rad_inc_oc=(0.25+0.5*n_sol/max_N)*rad_ext*(2.501-0.002361*temp);  # Radiacion solar incidente de onda corta (St) [MJm**-2d**-1]
        rad_noc=rad_inc_oc*(1-albedo);                                    # Radiacion neta de onda corta (Sn), ver valores de albedo en tablas
      elif resp==2:
        sys.exit("Te faltan demasiados datos, intentarlo nuevamente cuando se recolecten... GOOD LUCK!")  
      prompt_01 = 'Dispone de la temperatura al punto de rocio SI(1), NO (2) ? ';
      resp=int(input(prompt_01));
      if resp == 1:
        print('Tpr -> e -> emisividad neta')
        pvap=comp_e(Tpr)/10;          #en kPa             # Presion de vapor de saturacion con respecto al agua liquida [mb o hPa] input: T en °C 
        emi_neta=0.34-0.14*np.sqrt(pvap);                 # Emisividad neta
        # emi_neta=0.34-0.14*sqrt(Pr_vap_sat(Tpr)/10);
      elif resp==2:
        prompt_01 = 'Dispone de la temperatura minima SI(1), NO (2) ? ';
        resp=int(input(prompt_01)); 
        if resp == 1:
          print('tmin -> emisividad neta')
          emi_neta=0.34-0.14*np.sqrt(comp_e(Tmin)/10);   # Emisividad neta en funcion de tmin 
          # y esta funcion?
          #emi_neta=-0.02+0.261*exp(-7.77e-4*temp^2);    # Emisividad neta en funcion de la temp media del aire (°C)
        elif resp==2:
          print('tmedia -> emisividad neta')
          # emi_neta=0.34-0.14*sqrt(Pr_vap_sat(Tmin)/10);
          emi_neta=-0.02+0.261*np.exp(-7.77e-4*temp**2);
          #  exp(-7.77x10**-4*temp^2)

      
    # Coeficientes de nubosidad  
    # ac y bc: ac=1.35, bc=-0.35 (areas aridas)
    #          ac=1.0,  bc=0.0   (areas humedas)
    prompt_01 = 'El area de estudio es arida(1) o humeda (2) ? ';
    areash=int(input(prompt_01));
    if areash == 1:
      ac=1.35;
      bc=-0.35;
    elif areash == 2:
      ac=1;
      bc=0;
    print('rad para cielo claro -> f_nubosidad + emi_neta --> rnol +rnoc --> radmxd')
    rad_sol_cc=0.25*rad_ext*(2.501-0.002361*temp);
    # print('rad_sol_cc',rad_sol_cc);
    f_nubosidad=ac*(rad_inc_oc/rad_sol_cc)+bc;
    # print('f_nubo',f_nubosidad);
    rn_ol=f_nubosidad*emi_neta*4.903e-9*(temp+273.2)**4;
    # print('rnol',rn_ol)
    Rn_mmxd=(rad_noc+rn_ol)/(2.501-0.002361*temp);      # mm/dia
    # print('Rn_mmxd',Rn_mmxd)

  # Calculo deficit de presion a vapor
  print('Seleccione una de las sgtes opciones')
  print('Dispone de la temperatura maxima, temperatura minima y humedad relativa = 1');
  print('Dispone de la temperatura maxima, temperatura minima y temperatura media = 2');
  print('Dispone de la temperatura maxima, temperatura minima y temperatura al punto de rocio = 3');

  prompt_01 = 'Indique 1, 2 o 3 ? ';
  resp=int(input(prompt_01));

  if resp == 1:
  #  presion_vapor= comp_e(Tmax)/10;  
    defi=((comp_e(Tmax)+comp_e(Tmin))*0.5*(1-hr/100))/10;          # Entre 10 para pasar de mb a Kpa
  elif resp == 2:
    defi=((comp_e(Tmax)+comp_e(Tmin))*0.5-comp_e(temp)*hr/100)/10; # Kpa
  elif resp == 3:
    defi=((comp_e(Tmax)+comp_e(Tmin))*0.5-comp_e(Tpr))/10;         # Kpa  

  patm=101.3*((293-0.0065*alt)/293)**5.256;
  c_psi=(0.0016286*patm)/(2.501-0.002361*temp);
  delta=(4098*comp_e(temp)/10)/(237.3+temp)**2;
  c1=delta/(delta+c_psi);
  c2=(c_psi/(delta+c_psi))*(6.43*(1+0.536*vv))/(2.501-0.002361*temp);

  evap=c1*Rn_mmxd/(2.501-0.002361*temp)+c2*defi;

  print('Evaporacion (mm/dia)= ');
  print(evap);

# Descripcion de la funcion com_evap

Tiene datos de Radiacion neta [MJm-2d-1]?
  - SI
    --> calculamos Rad neta [mm/dia]  
  - NO
    - Tiene datos de Radiacion en onda corta [MJm-2d-1]?
      - SI
        --> calculamos Rad solar incidente en onda corta y posteriormente Rad. neta en onda corta con data de albedo
      - NO
        - Tiene datos de horas de sol?
          - SI
            --> calculamos max. horas de luz posible (N), max. rad. solar estraterrestre y en base a los coefientes de as y bs (en caso no tener coeficientes de angstrom) 
            --> calculamos Rad solar incidente en onda corta y posteriormente Rad. neta en onda corta con data de albedo
            - Tiene datos de Td?
              - SI
                --> calculamos la presion de vapor (ev) utilizando tetens y posteriormente la emisividad neta (e) con los coef. de emisividad 
                --> (*)
              - NO
                - Tiene datos de Tmin?
                  -SI
                    --> calculamos la presion de vapor (ev) utilizando tetens y posteriormente la emisividad neta (e) con los coef. de emisividad
                    --> (*)
                  - NO
                    calculamos la emisividad neta (e) con los coef. de emisividad y en funcion de la temperatura media
                    --> (*)
          - NO
            - Estime la evaporacion desde un tanque evaporimetro

      (*):  
      Se seleccionan los coef de nubosidad dependiendo si la zona es humeda o arida (ac y bc)  
      ac=1.35, bc=-0.35 (areas aridas)  
      ac=1.0,  bc=0.0   (areas humedas)  
      --> calculamos Rad. solar para cielo claro  
      --> calculamos el factor de nubosidad
      --> caldulamos Rad neta de onda larga
      --> calculamos Rad neta en MJm-2d-1 --> [mm/dia]  



Posteriormente Evaporacion utilizando el deficit de presion a vapor 
Para este ultimo paso necesita:
- tmax, tmin y humedad relativa
- tmax, tmin y tmean
- tmax, tmin y td

Finalmente:  
Convertimos Rn [mm/dia] en evaporacion utilizando el deficit de presion a vapor


In [ ]:
comp_evap(lat=lat, alt=alt, ano=ano, mes=mes, dia=dia, Rn=Rn, Roc=Roc, Rext=Rext, 
              temp=temp, Tpr=Tpr, Tmin=Tmin, Tmax=Tmax, n_sol=n_sol, albedo=albedo, hr=hr, vv=vv)

Dispone de la radiacion neta (MJm**-2d**-1) SI(1), NO (2) ? : 2
Dispone de la radiacion solar de onda corta (MJm**-2d**-1) SI(1), NO (2) ? 2
Dispone de registros de horas de sol SI(1), NO (2) ? 1
St(rad_inc_oc) -> Rnoc
Dispone de la temperatura al punto de rocio SI(1), NO (2) ? 2
Dispone de la temperatura minima SI(1), NO (2) ? 1
tmin -> emisividad neta
El area de estudio es arida(1) o humeda (2) ? 1
rad para cielo claro -> f_nubosidad + emi_neta --> rnol +rnoc --> radmxd
Seleccione una de las sgtes opciones
Dispone de la temperatura maxima, temperatura minima y humedad relativa = 1
Dispone de la temperatura maxima, temperatura minima y temperatura media = 2
Dispone de la temperatura maxima, temperatura minima y temperatura al punto de rocio = 3
Indique 1, 2 o 3 ? 2
Evaporacion (mm/dia)= 
7.3190638368328145


# testeos realizados

2,2,1,2,1..2 check  
2,2,1,2,2..2 check  
2,2,1,1,..2 check  
2,2,2 check   
1,...1/2/3 check   
2,1,2,2,...2 check  
2,1,2,1,...2 check  
2,1,1,.... 2 check  

## Codigo base en matlab


```
clear; close all; clc
% Calculo de la evaporacion
% Ref. Hidrologia Aplicada. Ven Te Chow
% Eduardo Chavarri V. SEPT - 2021
% Modificado MFigueroa 2021-II

% Informacion de entrada

[num, txt] = xlsread('data_evap.xlsx');
lat=num(1,1);    % Latitud
alt=num(1,2);    % Altitud (msnm)
ano=num(1,3);    % Anho
mes=num(1,4);    % Mes
dia=num(1,5);    % Dia
Rn=num(1,6);     % Radiacion neta (MJm-2d-1)
Roc=num(1,7);    % Radiacion onda corta (MJm-2d-1)
Rext=num(1,8);   % Radiacion extraterrestre (MJm-2d-1)
temp=num(1,9);   % Temperatura media (°C)
Tpr=num(1,10);   % Temperatura Punto Rocio (°C)
Tmin=num(1,11);  % Temperatura minima (°C)
Tmax=num(1,12);  % Temperatura maxima (°C)
n_sol=num(1,13); % Horas sol (h)
albedo=num(1,14);% Albedo 
hr=num(1,15);    % Humedad relativa (%)
vv=num(1,16);    % velocidad del viento a 2.0 de altura (ms-1)

nj=juliandate(ano,mes,dia);                 % Numero juliano
dec_sol=0.4093*sin((2*pi*nj/365)-1.405);    % Angulo de declinacion solar (radianes)
lat_rad=lat*pi/180;                         % Latitud en radianes
ang_atard=acos(-tan(lat_rad)*tan(dec_sol)); % Angulo de horas de atardecer
max_N=24*ang_atard/pi;                      % Maximas horas de luz
dist_sol=1+0.033*cos(2*pi*nj/365);          % Distancia relativa entre la tierra y el sol
rad_ext=15.392*dist_sol*(ang_atard*sin(lat_rad)*sin(dec_sol)+cos(lat_rad)*cos(dec_sol)*sin(ang_atard)); % Maxima radiacion solar extraterrestre

% Calculo energia disponible para la evaporacion

prompt_01 = 'Dispone de la radiacion neta (MJm-2d-1) SI(1), NO (2) ? ';
resp=input(prompt_01);

if resp == 1
  Rn_mmxd=Rn/(2.501-0.002361*temp);               % en mm/dia
else
  prompt_01 = 'Dispone de la radiacion solar de onda corta (MJm-2d-1) SI(1), NO (2) ? ';
  resp=input(prompt_01); 
  if resp == 1
    rad_inc_oc=Roc/(2.501-0.002361*temp) ;
    rad_noc=rad_inc_oc*(1-albedo); % Radiacion neta de onda corta
    prompt_01 = 'Dispone de la temperatura al punto de rocio SI(1), NO (2) ? ';
    resp=input(prompt_01);
    if resp == 1
      pvap=0.611*exp(((17.27*Tpr)/(237.3+Tpr)));  % Presion de vapor
      emi_neta=0.34-0.14*sqrt(pvap);              % Emisividad neta
    else
      prompt_01 = 'Dispone de la temperatura minima SI(1), NO (2) ? ';
      resp=input(prompt_01); 
      if resp == 1
       emi_neta=0.34-0.14*sqrt(Pr_vap_sat(Tmin)/10);
      else
       emi_neta=-0.02+0.261*exp(-7.77e-4*temp^2); 
      end    
    end      
    
  else
    prompt_01 = 'Dispone de registros de horas de sol SI(1), NO (2) ? ';
    resp=input(prompt_01);
    if resp == 1
%       rad_inc_oc=(0.25+0.5*n_sol/max_N)* rad_ext*(2.501-0.002361*temp); 
%       rad_noc=rad_inc_oc*(1-albedo);
      
      % Coeficientes de nubosidad
      % ac y bc: ac=1.35, bc=-0.35 (areas aridas)
      %          ac=1.0,  bc=0.0   (areas humedas)
      ac=1.35;
      bc=-0.35;
      %  ac=1;
      %  bc=0;      
%       rad_sol_cc=0.25*rad_ext*(2.501-0.002361*temp);                    % Radiacion solar para cielo claro
      rad_inc_oc=(0.25+0.5*n_sol/max_N)* rad_ext*(2.501-0.002361*temp);
      rad_noc=rad_inc_oc*(1-albedo);
%       f_nubosidad=ac*(rad_inc_oc/rad_sol_cc)+bc;                        % Factor de nubosidad
      %f_nubosidad=1;
%       rn_ol=f_nubosidad*emi_neta*4.903e-9*(temp+273.2)^4;               % Radiacion neta de onda larga
%       Rn_mmxd=(rad_noc+rn_ol)/(2.501-0.002361*temp);                    % Radiacion neta total mm/dia

      
    else
      return; %break  
    end    
    prompt_01 = 'Dispone de la temperatura al punto de rocio SI(1), NO (2) ? ';
    resp=input(prompt_01);
    if resp == 1
      emi_neta=0.34-0.14*sqrt(Pr_vap_sat(Tpr)/10);
    else
      prompt_01 = 'Dispone de la temperatura minima SI(1), NO (2) ? ';
      resp=input(prompt_01); 
      if resp == 1
       emi_neta=0.34-0.14*sqrt(Pr_vap_sat(Tmin)/10); 
      else
       emi_neta=-0.02+0.261*exp(-7.77e-4*temp^2);
      end
    end 
    
  end
  
  % Coeficientes de nubosidad  
  % ac y bc: ac=1.32, bc=-0.35 (areas aridas)
  %          ac=1.0,  bc=0.0   (areas humedas)
  
  ac=1.35;
  bc=-0.35;
  %  ac=1;
  %  bc=0;
  rad_sol_cc=0.25*rad_ext*(2.501-0.002361*temp);
  f_nubosidad=ac*(rad_inc_oc/rad_sol_cc)+bc;
  % f_nubosidad=1;
  rn_ol=f_nubosidad*emi_neta*4.903e-9*(temp+273.2)^4;
  Rn_mmxd=(rad_noc+rn_ol)/(2.501-0.002361*temp);      % mm/dia

end

% Calculo deficit de presion a vapor
disp('Dispone de la temperatura maxima, temperatura minima y humedad relativa = 1');
disp('Dispone de la temperatura maxima, temperatura minima y temperatura media = 2');
disp('Dispone de la temperatura maxima, temperatura minima y temperatura al punto de rocio = 3');

prompt_01 = 'Indique 1, 2 o 3 ? ';
resp=input(prompt_01);

if resp == 1
 %presion_vapor= Pr_vap_sat(Tmax)/10;  
 def=((Pr_vap_sat(Tmax)+Pr_vap_sat(Tmin))*0.5*(1-hr/100))/10;               % Entre 10 para pasar de mb a Kpa
end

if resp == 2
 def=((Pr_vap_sat(Tmax)+Pr_vap_sat(Tmin))*0.5-Pr_vap_sat(temp)*hr/100)/10; % Kpa
end

if resp == 3
 def=((Pr_vap_sat(Tmax)+Pr_vap_sat(Tmin))*0.5-Pr_vap_sat(Tpr))/10;         % Kpa  
end

patm=101.3*((293-0.0065*alt)/293)^5.256;
c_psi=(0.0016286*patm)/(2.501-0.002361*temp);
delta=(4098*Pr_vap_sat(temp)/10)/(237.3+temp)^2;
c1=delta/(delta+c_psi);
c2=(c_psi/(delta+c_psi))*(6.43*(1+0.536*vv))/(2.501-0.002361*temp);

evap=c1*Rn_mmxd/(2.501-0.002361*temp)+c2*def;

disp ('Evaporacion (mm/dia)= ');
disp (evap);
```